In [1]:
import tensorflow as tf
import numpy as np
import struct
import gzip
# import matplotlib
import matplotlib.pyplot as plt
from array import array

from flatbuffers.packer import float32
from tensorflow.python.data.experimental.ops.distribute import batch_sizes_for_worker

# my project
from module.conf import PROJECT_DIR
# matplotlib.use("QTAgg")
%matplotlib inline

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [4]:
class Generator(tf.keras.Model):
    def __init__(self, noise_dim: int, target_shape: tuple):
        super(Generator, self).__init__()
        self.model = tf.keras.Sequential(layers= [
            tf.keras.layers.InputLayer(input_shape=(noise_dim,)),
            tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=(target_shape[0] * target_shape[1]), activation=tf.nn.tanh),
            tf.keras.layers.Reshape(target_shape=target_shape)
        ])
        return

    def call(self, inputs, training=False):
        return self.model(inputs, training=training)

class Discriminator(tf.keras.Model):
    def __init__(self, target_shape: tuple):
        super(Discriminator, self).__init__()
        self.model = tf.keras.Sequential(layers=[
            # tf.keras.layers.InputLayer(shape=target_shape),
            tf.keras.layers.Flatten(input_shape=target_shape),
            tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid),
        ])

    def call(self, inputs, training=False):
        return self.model(inputs, training=training)


In [5]:
class GAN(tf.keras.Model):
    def __init__(self, noise_dim=100, target_shape=[28, 28]):
        super(GAN, self).__init__()
        self.gen_optimizer = None
        self.disc_optimizer = None
        self.loss_fn = None
        self.noise_dim = noise_dim
        self.target_shape = target_shape
        self.generator = Generator(self.noise_dim, self.target_shape)
        self.discriminator = Discriminator(self.target_shape)

    def compile(self, gen_optimizer, disc_optimizer, loss_fn):
        super(GAN, self).compile()
        self.gen_optimizer = gen_optimizer
        self.disc_optimizer = disc_optimizer
        self.loss_fn = loss_fn
        pass

    def train_step(self, batch):
        batch_sizes = tf.shape(batch)[0]
        # random noise
        noise = tf.random.normal(shape=(batch_sizes, self.noise_dim))
        # gen fake img by Generator
        fake_images = self.generator(noise, training=True)
        # 1 - real image, 0 - fake image
        real_labels = tf.ones((batch_sizes, 1))
        fake_labels = tf.zeros((batch_sizes, 1))

        # train Discriminator
        with tf.GradientTape() as disc_tape:
            real_preds = self.discriminator(batch, training=True)
            fake_preds = self.discriminator(fake_images, training=True)

            real_loss = self.loss_fn(real_labels, real_preds)
            fake_loss = self.loss_fn(fake_labels, fake_preds)

            disc_loss = (real_loss + fake_loss) / 2
            pass
        grad = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        self.disc_optimizer.apply_gradients(zip(grad, self.discriminator.trainable_variables))

        # train Generator
        with tf.GradientTape() as gen_tape:
            fake_images = self.generator(noise, training=True)
            fake_preds = self.discriminator(fake_images, training=True)
            gen_loss = self.loss_fn(real_labels, fake_preds)
            pass
        grad = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        self.gen_optimizer.apply_gradients(zip(grad, self.generator.trainable_variables))
        return {"gen_loss": gen_loss, "disc_loss": disc_loss}

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(60000).batch(128)
(x_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
x_train = (x_train.astype("float64") - 127.5) / 127.5  # Standartize to [-1, 1]

In [10]:
x_train.shape[0]

50000

In [14]:
# dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(x_train.shape[0]).batch(128)
dataset = x_train

In [15]:
gan = GAN(noise_dim=512)

gan.compile(
    # gen_optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    # disc_optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    # loss_fn=tf.keras.losses.BinaryCrossentropy()
    # gen_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9),
    # disc_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9),
    # loss_fn=tf.keras.losses.BinaryCrossentropy()
    gen_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    disc_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy()
)

# gan.fit(dataset, epochs=10000)
gan.fit(dataset, epochs=500, batch_size=256)

Epoch 1/500


ValueError: in user code:

    c:\Users\nguyenqh\tools\miniconda3\envs\learn-ml\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\nguyenqh\AppData\Local\Temp\ipykernel_16112\1540437820.py:36 call  *
        return self.model(inputs, training=training)
    c:\Users\nguyenqh\tools\miniconda3\envs\learn-ml\lib\site-packages\keras\engine\base_layer.py:1037 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    c:\Users\nguyenqh\tools\miniconda3\envs\learn-ml\lib\site-packages\keras\engine\sequential.py:369 call
        return super(Sequential, self).call(inputs, training=training, mask=mask)
    c:\Users\nguyenqh\tools\miniconda3\envs\learn-ml\lib\site-packages\keras\engine\functional.py:414 call
        return self._run_internal_graph(
    c:\Users\nguyenqh\tools\miniconda3\envs\learn-ml\lib\site-packages\keras\engine\functional.py:550 _run_internal_graph
        outputs = node.layer(*args, **kwargs)
   

In [ ]:
gan.save_weights(PROJECT_DIR + "/data/models/cifar10_gan_500/model")

In [ ]:
# gan.load_weights(PROJECT_DIR + "/data/models/fashionmnist_gan_1000_2/model")

In [ ]:
import matplotlib.pyplot as plt

def generate_and_show_images(generator, num_examples=10):
    noise = tf.random.normal(shape=(num_examples, 512))
    images = generator(noise, training=False)
    images = (images + 1) / 2.0  # standardize to [0, 1]
    rows=4

    fig, axes = plt.subplots(nrows=rows, ncols=num_examples//rows, figsize=(10, 5))
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i], cmap="gray")
        ax.axis("off")
    plt.show()

generate_and_show_images(gan.generator, num_examples=20)